In [1]:
# keras module for building LSTM
from keras.utils import pad_sequences
from keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional
from keras.preprocessing.text import Tokenizer
from keras.callbacks import EarlyStopping
from keras.models import Sequential
import keras.utils as ku

# set seeds for reproducability
import tensorflow as tf

import pandas as pd
import numpy as np
import string, os
import json as pjson
import warnings
import matplotlib.pyplot as plt
from keras.preprocessing.text import tokenizer_from_json
warnings.filterwarnings("ignore")
warnings.simplefilter(action="ignore", category=FutureWarning)

from dataset import load_tokenized_sentences

sentence_len_min_limit = 2
sentence_len_max_limit = 25

dataset1 = load_tokenized_sentences("../datasets/pickled/books_clear.pickle") 
dataset2 = load_tokenized_sentences("../datasets/pickled/bajki-extebd_clear.pickle") 
dataset = dataset1 + dataset2
print(len(dataset), dataset[:5])

dataset_join = []
for sentence in dataset:
    if len(sentence) > sentence_len_max_limit:
        for i in range(0, len(sentence) - sentence_len_min_limit, sentence_len_max_limit):
           dataset_join.append(" ".join(sentence[i:i+sentence_len_max_limit])) 
    if sentence_len_min_limit <= len(sentence) <= sentence_len_max_limit:
        dataset_join.append(" ".join(sentence))
dataset = dataset_join

print(len(dataset), dataset[:5])

588313 [['jules', 'verne', 'mil', 'podmorskiej', 'żeglugitłum'], ['tłumacz', 'nieznanyisbn', 'skała', 'uciekającanie', 'zapomniano', 'zapewne', 'dotąd', 'wypadku', 'dziwnego', 'niepojętego', 'i', 'trudnego', 'do', 'objaśnienia', 'zjawiska', 'jakim', 'się', 'odznaczył', 'rok'], ['nie', 'mówiąc', 'już', 'o', 'pogłoskach', 'niepokojących', 'ludność', 'portów', 'i', 'zajmujących', 'ogół', 'na', 'wszystkich', 'lądach', 'dodać', 'wypada', 'że', 'marynarze', 'byli', 'najmocniej', 'zaniepokojeni'], ['kupcy', 'armatorzy', 'dowódcy', 'okrętów', 'szyprowie', 'i', 'sternicy', 'statków', 'europejskich', 'i', 'amerykańskich', 'oficerowie', 'marynarki', 'wojennej', 'wszystkich', 'krajów', 'a', 'nawet', 'rządy', 'różnych', 'państw', 'obu', 'lądów', 'do', 'najwyższego', 'stopnia', 'zajęci', 'byli', 'tym', 'wydarzeniem'], ['od', 'niejakiego', 'czasu', 'okręty', 'napotykały', 'na', 'morzu', 'jakąś', 'rzecz', 'ogromną', 'przedmiot', 'długi', 'kształtu', 'wrzecionowatego', 'niekiedy', 'świecący', 'nieskońc

In [2]:
from typing import List


def get_sequence_of_tokens(dataset: List[str], tokenizer: Tokenizer = None):
    if tokenizer is None:
        tokenizer = Tokenizer()
        tokenizer.fit_on_texts(dataset)

    total_words = len(tokenizer.word_index) + 1
    word2int_sequences = []
    for line in dataset:
        token_list = tokenizer.texts_to_sequences([line])[0]
        for i in range(sentence_len_min_limit, len(token_list)):
            n_gram_sequence = token_list[: i + 1]
            word2int_sequences.append(n_gram_sequence)
    return word2int_sequences, total_words, tokenizer


def generate_padded_sequences(word2int_sequences: List[List[int]]):
    max_sequence_len = max([len(x) for x in word2int_sequences])
    word2int_sequences = np.array(
        pad_sequences(word2int_sequences, maxlen=max_sequence_len, padding="pre")
    )
    predictors, label = word2int_sequences[:, :-1], word2int_sequences[:, -1]
    return predictors, label, max_sequence_len


def dataset_generator(predictors, label, batch_size = 256, shuffle = True):
    while True:
        if shuffle:
            p = np.random.permutation(len(predictors))
        else:
            p = np.arange(len(predictors))
        for i in range(0, len(predictors) - batch_size + 1, batch_size):
            indexes = p[i : i + batch_size]
            yield predictors[indexes], label[indexes]

with open('../lstm_models/tokenizer.json', 'r') as f:
    data = pjson.load(f)
    tokenizer = tokenizer_from_json(data)
    
word2int_sequences, total_words, tokenizer = get_sequence_of_tokens(dataset, tokenizer)
print(word2int_sequences[:10])

predictors, label, max_sequence_len = generate_padded_sequences(word2int_sequences)
print(predictors.shape, label.shape)

batch_size = 256
gen = dataset_generator(predictors, label, 256, True)
a, b = next(gen)
print(a.shape, b.shape)

[[62859, 103749, 1245], [62859, 103749, 1245, 33519], [62859, 103749, 1245, 33519, 171004], [21370, 88416, 9660], [21370, 88416, 9660, 171005], [21370, 88416, 9660, 171005, 13608], [21370, 88416, 9660, 171005, 13608, 532], [21370, 88416, 9660, 171005, 13608, 532, 539], [21370, 88416, 9660, 171005, 13608, 532, 539, 1731], [21370, 88416, 9660, 171005, 13608, 532, 539, 1731, 2222]]
(6483631, 24) (6483631,)
(256, 24) (256,)


In [3]:
import random
from keras import optimizers

class PredictCallback(tf.keras.callbacks.Callback):
    def __init__(self, seed_text, next_words, max_sequence_len, randomize=True, max_randomize_words = 2):
        self.seed_text = seed_text
        self.next_words = next_words
        self.max_sequence_len = max_sequence_len
        self.randomize = randomize
        self.max_randomize_words = max_randomize_words

    def on_epoch_begin(self, epoch, logs=None):
        seed_text: str = self.seed_text
        for _ in range(self.next_words):
            token_list = tokenizer.texts_to_sequences([seed_text])[0]
            token_list = pad_sequences(
                [token_list], maxlen=self.max_sequence_len - 1, padding="pre"
            )
            predicted = self.model.predict_on_batch(token_list)[0]
            if self.randomize:
                indc = np.argpartition(predicted, -self.max_randomize_words)[-self.max_randomize_words:]
                predicted = random.choice(indc)
            else:
                predicted = predicted.argmax()
            
            output_word = ""
            for word, index in tokenizer.word_index.items():
                if index == predicted:
                    output_word = word
                    break
            seed_text += " " + output_word
        print(f"Start epoch {epoch} of training; Generated text:", seed_text)


def create_model(max_sequence_len: int, total_words: int) -> Sequential:
    input_len = max_sequence_len - 1
    model = Sequential()
    model.add(Embedding(total_words, 50, input_length=input_len))
    model.add(LSTM(128, return_sequences=True))
    model.add(Dropout(0.2))
    model.add(LSTM(256, return_sequences=True))
    model.add(LSTM(512))
    model.add(Dense(total_words, activation="softmax"))
    model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizers.Adam(1e-4))
    return model

print(total_words, max_sequence_len)
    
model = create_model(max_sequence_len, total_words)
model.load_weights('../lstm_models/model_best_2.h5')
for layer in model.layers[:-2]:
    layer.trainable = False
model.summary()



331307 25
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 24, 50)            16565350  
                                                                 
 lstm (LSTM)                 (None, 24, 128)           91648     
                                                                 
 dropout (Dropout)           (None, 24, 128)           0         
                                                                 
 lstm_1 (LSTM)               (None, 24, 256)           394240    
                                                                 
 lstm_2 (LSTM)               (None, 512)               1574912   
                                                                 
 dense (Dense)               (None, 331307)            169960491 
                                                                 
Total params: 188,586,641
Trainable params: 17

In [4]:
model.fit(
    gen,
    steps_per_epoch=len(predictors) // batch_size,
    epochs=10,
    verbose=1,
    callbacks=[
        PredictCallback("dawno temu czerwony kapturek poszedł do lasu", 25, max_sequence_len, True),
        PredictCallback("dawno temu czerwony kapturek poszedł do lasu", 25, max_sequence_len, False),
        tf.keras.callbacks.ModelCheckpoint('../lstm_models/model_best_2_retrained.h5', monitor='loss', save_best_only=True, save_weights_only=True)
    ],
)

Start epoch 0 of training; Generated text: dawno temu czerwony kapturek poszedł do lasu ale już już dawno było coś bolesnego i nieustającym w tym celu że nie mógł już sobie dać rady z resztą czasu i sam z
Start epoch 0 of training; Generated text: dawno temu czerwony kapturek poszedł do lasu i już nie było nikogo kto by się nie zapamiętał i nie mógł pojąć począł krzyczeć wniebogłosy bo nie mógł się powstrzymać od uniesienia i
Epoch 1/10
25326/25326 [==============================] - 1760s 69ms/step - loss: 3.9509
Start epoch 1 of training; Generated text: dawno temu czerwony kapturek poszedł do lasu a potem położyła ją na stole odwinęła z chustki garronego i drżącymi rękami nie chciało się mówić padłem na nich i pocałowała go za ramię
Start epoch 1 of training; Generated text: dawno temu czerwony kapturek poszedł do lasu i uśmiechała się do mnie z gorącą uciechą i z przejęciem w spojrzeniu przyczem gdy się już nie dotarło do tego policzka połyskiwały sine rumieńce
Epoch 2/10
25326/2532

In [6]:
model.load_weights('../lstm_models/model_best_2_retrained.h5')

texts = [
    "dawno temu czerwony kapturek poszedł do lasu",
    "sierotka weszła do domu i zobaczyła siedzącą przy stolę babcię",
    "była sobie zła królowa która chciała panować nad światem"
]
for text in texts: 
    callb = PredictCallback(text, 500, max_sequence_len, True)
    callb.model = model
    callb.on_epoch_begin(0)
    

Start epoch 0 of training; Generated text: dawno temu czerwony kapturek poszedł do lasu i już nie mogła się oprzeć wrażeniu i uniosła go do ziemi a ona nie mogła się jej z tym pogodzić na ziemię i na oślep do domu a potem w południe i w tej samej izbie z drugiej z nich pasło się w gęstwinę głazach w zielonych gniazdach się z nim rosła a na końcu królewskim rosła i przygotowywano w górę a ziarno w lesie stuknęło jak obłoki jęło w spienioną ulewę i w końcu się dzieje i wrzawa dzwonów i organy żałośnie chciały i w ręku się cisną a w nich się z nią wszystko co ściągnęło ramionami a potem zmykaj mi na pomoc i wracaj do mnie i powiedz mi wszystko co się stało w tej komnacie i że nie mogę ci dać rady ani na moment bo to nie ma nic przeciwko temu że to jest wróżba i że w tej chwili jest ci się niegodni i że nie ma ani grosza ani na gorycz pierścienia i nie może być z tego powodu że to jest dla mnie obojga co do tego co się nazywa i z tego nie mam nic wspólnego ze sobą nie mogę być moim mężem a